In [2]:
import pennylane as qml
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
wires = range(15)
dev = qml.device('default.qubit', wires=wires)

In [4]:
data = np.load('data/prices_15_4.npy', allow_pickle=True)

In [5]:
msft_data = data[0]
msft_b = msft_data[0]
msft_f = msft_data[1]

In [6]:
def reshape_to_one_axis(data):
    in_one_arr = []
    for i in range(19):
        if i <= 14:
            in_one_arr.append(data[0][i])
        elif i >=15:
            in_one_arr.append(data[1][i-15])
    return np.reshape(in_one_arr, (1,1,19))

## Structure of QGAN

### Generator

In [7]:
def gen_ansatz(w):
    qml.broadcast(unitary=qml.RY, pattern = 'single', wires = wires, parameters = w[0:15])
    for k in range(1, int(len(w)) // 15):
        qml.broadcast(unitary=qml.CZ, pattern = 'ring', wires=wires)
        qml.broadcast(unitary=qml.RY, pattern = 'single', wires = wires, parameters = w[(15*k):(15*(k+1))])

In [8]:
@qml.qnode(dev, interface="tf")
def gen_circuit(b_seq, gen_weights):
    qml.templates.AngleEmbedding(b_seq, wires, rotation='X')
    gen_ansatz(gen_weights)
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

In [9]:
def generator(equity_data, gen_weights):
    generated_prices =  gen_circuit(equity_data[0], gen_weights).numpy()
    return [equity_data[0], gen_circuit(equity_data[0], gen_weights).numpy()]

In [10]:
init_gen_weights = np.random.normal(size=(15,))

In [11]:
tf.Variable(init_gen_weights)

<tf.Variable 'Variable:0' shape=(15,) dtype=float64, numpy=
array([-0.12884513, -0.01361892,  0.86088683, -1.04190869, -1.13304563,
       -0.07415381,  0.32951867,  0.32369006,  0.17095212,  0.07537326,
       -2.04152151,  1.29285969, -0.34701718,  0.41861991,  0.59777649])>

In [12]:
generator(msft_data, init_gen_weights)

[array([-1.        ,  0.68858394, -0.71996818,  0.64508565,  0.12111897,
         0.84157581,  0.8777076 ,  0.22345133, -0.30409296, -0.18410706,
         0.        , -0.8391851 ,  0.06801392,  0.02368372,  0.0141945 ]),
 array([0.53582371, 0.77207501, 0.49001443, 0.4031784 ])]

In [13]:
"""Testing function to retrieve gradients from generator"""
def gen_grad(data, w):
    with tf.GradientTape() as tape:
        loss = gen_circuit(msft_data[0], w)
    gen_gradient = tape.gradient(loss, w)
    print(w.numpy(), gen_gradient)

In [14]:
gen_grad(msft_data, tf.Variable(init_gen_weights))

[-0.12884513 -0.01361892  0.86088683 -1.04190869 -1.13304563 -0.07415381
  0.32951867  0.32369006  0.17095212  0.07537326 -2.04152151  1.29285969
 -0.34701718  0.41861991  0.59777649] tf.Tensor(
[ 0.06942287  0.01051548 -0.57020106  0.68987341  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.        ], shape=(15,), dtype=float64)


### Discriminator

In [63]:
def make_discrim():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=16, kernel_size=1, activation='relu', input_shape=[1, 19]))
    model.add(tf.keras.layers.Conv1D(filters=16, kernel_size=1, activation='relu'))
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    
    return model

In [64]:
discriminator = make_discrim()
discriminator.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_13 (Conv1D)           (None, 1, 16)             320       
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 1, 16)             272       
_________________________________________________________________
dense_12 (Dense)             (None, 1, 128)            2176      
_________________________________________________________________
dense_13 (Dense)             (None, 1, 1)              129       
Total params: 2,897
Trainable params: 2,897
Non-trainable params: 0
_________________________________________________________________


## Pennylane construction of loss

In [17]:
def prob_real_true(equity_data):
    equity_data_in_one_dim = reshape_to_one_axis(equity_data)
    confidence = discriminator(equity_data_in_one_dim).numpy()[0][0][0]
    prob_real_true = (confidence + 1)/ 2
    return prob_real_true

In [18]:
def prob_fake_true(equity_data, gen_weights):
    gen_data = [equity_data[0], gen_circuit(equity_data[0], gen_weights).numpy()]
    gen_data_in_one_dim = reshape_to_one_axis(gen_data)
    confidence = discriminator(gen_data_in_one_dim).numpy()[0][0][0]
    prob_fake_true = (confidence + 1)/ 2
    return prob_fake_true

In [19]:
"""Test"""
genn = generator(data[2], init_gen_weights)
print(prob_fake_true(genn, init_gen_weights))
print(prob_real_true(data[2]))

0.7634900510311127
0.7557262778282166


In [30]:
def gen_cost(gen_weights):
    return -prob_fake_true(msft_data, gen_weights)

def disc_cost():
    cost = prob_fake_true(msft_data, gen_weights) - prob_real_true(msft_data)
    return cost

In [44]:
gen_weights = init_gen_weights
try:
    with tf.GradientTape() as tape:
        gen_cost = gen_cost(gen_weights)
    print(tape.gradient(gen_cost, gen_weights))
except:
    print("this block go bad")

this block go bad


## Loss — real is 1, fake is 0

In [36]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    """Calculating loss"""
    return cross_entropy(np.ones_like(fake_output), fake_output)

In [37]:
def discriminator_loss(fake_output, real_output):    
    """Compute discriminator loss.""" 
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [38]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [93]:
BATCH_SIZE=256


def train_step(equity_data, gen_weights):
    """Run train step on provided image batch."""
    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape: 
        generated_prices = [equity_data[0], gen_circuit(equity_data[0], gen_weights)] 
        
        """Reshaping equity arrays to feed into discrim"""
        gen_prices_in_one_arr = reshape_to_one_axis(generated_prices)
        real_prices_in_one_arr = reshape_to_one_axis(equity_data)

        """Getting outputs from discrim"""
        real_output = discriminator(real_prices_in_one_arr)
        fake_output = discriminator(gen_prices_in_one_arr)

        """Calculating loss"""
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, gen_weights)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    print(gradients_of_generator)
#     generator_optimizer.apply_gradients(
#         zip(gradients_of_generator, gen_weights))
#     discriminator_optimizer.apply_gradients(
#         zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [94]:
train_step(msft_data, tf.Variable(init_gen_weights))

None


(<tf.Tensor: shape=(), dtype=float32, numpy=0.7042643>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.3934547>)

## Testing new pipeline with just f_seq

### Discriminator

In [82]:
def make_disc():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv1D(filters=16, kernel_size=1, activation='relu', input_shape=[1, 4]))
    model.add(tf.keras.layers.Conv1D(filters=16, kernel_size=1, activation='relu'))
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    
    return model

In [83]:
small_disc = make_disc()
small_disc.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_22 (Conv1D)           (None, 1, 16)             80        
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 1, 16)             272       
_________________________________________________________________
dense_20 (Dense)             (None, 1, 128)            2176      
_________________________________________________________________
dense_21 (Dense)             (None, 1, 1)              129       
Total params: 2,657
Trainable params: 2,657
Non-trainable params: 0
_________________________________________________________________


In [84]:
small_disc(gen_circuit(msft_data[0], init_gen_weights))

ValueError: Input 0 of layer sequential_12 is incompatible with the layer: : expected min_ndim=3, found ndim=1. Full shape received: (4,)

In [40]:
BATCH_SIZE=256


def train_step(equity_data, gen_weights):
    """Run train step on provided image batch."""
    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape: 
        generated_prices = gen_circuit(equity_data[0], gen_weights)
        

        """Getting outputs from discrim"""
        real_output = discriminator(real_prices_in_one_arr)
        fake_output = discriminator(gen_prices_in_one_arr)

        """Calculating loss"""
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

#     with tf.GradientTape() as gen_tape:
#         f_seq = gen_circuit(equity_data[0], gen_weights)
#         generated_prices = [equity_data[0], f_seq]
#         loss = generator_loss(discriminator(reshape_to_one_axis(generated_prices)))
#     gen_gradient = gen_tape.gradient(f_seq, gen_weights)
    
#     print(gen_gradient)
    
    gradients_of_generator = gen_tape.gradient(gen_loss, gen_weights)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    print(gradients_of_generator)
#     generator_optimizer.apply_gradients(
#         zip(gradients_of_generator, gen_weights))
#     discriminator_optimizer.apply_gradients(
#         zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [41]:
train_step(msft_data, tf.Variable(init_gen_weights))

None


(<tf.Tensor: shape=(), dtype=float32, numpy=0.61227745>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.3528016>)